# Auto Machine Learning with H20 Auto ML

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import h2o        
from h2o.automl import H2OAutoML

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/test.csv')

samp_sub = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv')

In [ ]:
display('train data')
display(train_df.shape)
display(train_df.head())
print('#############################################')
display('test data')
display(test_df.shape)
display(test_df.head())

In [ ]:
h2o.init()

In [ ]:
train_h2o = h2o.H2OFrame(train_df)
test_h2o = h2o.H2OFrame(test_df)

In [ ]:
train, test = train_h2o.split_frame(ratios = [0.75])

x = train.columns
y = 'failure'
x.remove(y)

In [ ]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=20,
                max_runtime_secs=600,
                exclude_algos=["DeepLearning"],
                seed=42)

%time aml.train(x=x, y=y, training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
preds = aml.leader.predict(test_h2o)
final = preds.cbind(test_h2o)
final

In [ ]:
submission_df = final[['id','p1']]
submission_df.set_names(['id', 'failure'])
sub_df = submission_df.as_data_frame(use_pandas=True, header=True)
sub_df['failure'] = sub_df['failure'].astype('float')
sub_df.head()
sub_df.to_csv('submission.csv', index=False)